In [6]:
import ROOT
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from matplotlib.ticker import MaxNLocator, AutoMinorLocator

# PARAMETERS
---

In [ ]:
fsize = 15
tsize = 18
tdir = 'inout'
major = 5.0
minor = 3.0
lwidth = 0.8
lhandle = 2.0
plt.style.use('default')
plt.rcParams['text.usetex'] = True
plt.rcParams['font.size'] = fsize
plt.rcParams['legend.fontsize'] = tsize-4
plt.rcParams['xtick.direction'] = tdir
plt.rcParams['ytick.direction'] = tdir
plt.rcParams['xtick.major.size'] = major
plt.rcParams['xtick.minor.size'] = minor
plt.rcParams['ytick.major.size'] = 5.0
plt.rcParams['ytick.minor.size'] = 3.0
plt.rcParams['axes.linewidth'] = lwidth
plt.rcParams['legend.handlelength'] = lhandle

sigbg_color = 'r'
sig_color = 'g'
bg_color = 'b'

sigbg_marker = '^'
sig_marker = 'o'
bg_marker = 'v'

sigbg_fmt = sigbg_color+sigbg_marker
sig_fmt = sig_color+sig_marker
bg_fmt = bg_color+bg_marker

plt.rcParams['text.usetex'] = True

methods=["sideband","splot"]
regions=["sig","sigbg","bg"]
fitpars=["\sin(2\phi_{h}-2\phi_{R})","\sin(\phi_{h}-\phi_{R})", "\sin(-\phi_{h}+2\phi_{R})" , "\sin(\phi_{R})", "\sin(\phi_{h})", "\sin(2\phi_{h}-\phi_{R})" , "\sin(3\phi_{h}-2\phi_{R})"]

In [11]:
xDict = {
    "name"   : "x",
    "xtitle" : r"$x_{B}$",
    "xticks" : np.arange(0,0.6,0.1)
}
zDict = {
    "name"   : "z",
    "xtitle" : r"$z_{h}$",
    "xticks" : np.arange(0.3,1,0.1)
}

MhDict = {
    "name"   : "Mh",
    "xtitle" : r"$M_{h}$",
    "xticks" : np.arange(0.2,1.6,0.2)
}

dicts = [xDict,zDict,MhDict]

# HELPER FUNCTIONS
---

In [4]:
# Return the files relevant for the asymmetry plotting
def get_files(filedir = ""):
    asym_file = filedir+"asymmetry.root"
    sideband_files = [str(path) for path in Path(filedir).glob("*sideband*/*.root")]
    splot_files = [str(path) for path in Path(filedir).glob("*splot*/*.root")]

    return asym_file, sideband_files, splot_files

In [ ]:
def get_substr(string,sub1,sub2):
    # getting index of substrings
    idx1 = string.index(sub1)
    idx2 = idx1+len(sub1)+string[idx1+len(sub1):].index(sub2)
    res = string[idx1 + len(sub1): idx2]
    return res

# By reading in the file names, figure out what binnings need to be used
def get_bins(files):
    
    
    # Get bin types
    bintypes=[]
    for file in files:
        res = get_substr(file,"bin_","_")
        if(not res in bintypes):
            bintypes.append(res)
            
    # Get number of bins for each bintype
    bins_per_bintype = [0 for i in range(len(bintypes))]
    for file in files:
        res = get_substr(file,"bin_","_")
        bintype_idx = bintypes.index(res)
        bins_per_bintype[bintype_idx]+=1
        

    # Get bin ranges
    bins = [[] for i in range(len(bintypes))]
    for file in files:
        bintype = ""
        # Which bin type does this file correspond to?
        for bt in bintypes:
            if("_"+bt+"_" in file):
                bintype = bt
        
        binleft = get_substr(file,"_"+bintype+"_","_")
        binright = get_substr(file,"_"+binleft+"_","/")
        
        bintype_idx = bintypes.index(bintype)
        
        if(not float(binleft) in bins[bintype_idx]):
            bins[bintype_idx].append(float(binleft))
        if(not float(binright) in bins[bintype_idx]):
            bins[bintype_idx].append(float(binright))
            
    # Sort binnings
    for i,bArr in zip(range(len(bintypes)),bins):
        before = np.array(bArr)
        after = np.sort(before)
        bins[i] = after.tolist()

    return bintypes,bins

In [ ]:
# Grab the .root file based on binning
def get_rootfile(files,
              bintype = "",
              bl = 0,
              br = 0,
              BIN = 0):
    if(BIN):
        if(bl or br):
            print("ERROR: get_rootfile() cannot be passed both a BIN and bl+br...only doing BIN")
        bl=BIN[0]
        br=BIN[1]
    
    for file in files:
        if(not "_"+bintype+"_" in file):
            continue
        if(not "_"+str(bl) in file):
            continue
        if(not "_"+str(br) in file):
            continue
        else:
            return file

In [ ]:
def get_tge_from_asym(asym_file = "",
                      m = "",
                      bt = "",
                      r = "",
                      idx = ""):
    tfile = ROOT.TFile(asym_file,"READ")
    tlist = tfile.GetListOfKeys()
    tnames = [tobj.GetName() for tobj in tlist]
    for tname in tnames:
        if((m in tname) and ("_"+bt+"_" in tname) and ("_"+r+"_" in tname) and ("_"+idx in tname)):
            tge = tfile.Get(tname)
            return tge.Clone()
    print("ERROR: get_tge_from_asym() found no TObject match in asymmetry.root...Aborting...")
    return -1

In [ ]:
# Using the bin left edge and bin right edge, find the correct data point in the TGraphErrors
def get_v_e(tge,bl,br):
    for i in range(tge.GetN()):
        if(bl<=tge.GetPointX(i) and br>=tge.GetPointX(i)):
            return tge.GetPointY(i),tge.GetErrorY(i)
    print("ERROR: get_v_e could not find a data point between",bl,"and",br,"in TGraphErrors",tge.GetName(),"...Aborting...")
    return 0,0

# MAIN CODE
---

In [12]:
def unpack_asym(filedir = ""):
    # First, grab the relevant files
    asym_file, sideband_files, splot_files = get_files(filedir)
    # Extract binnings from data set
    bintype_Arr, bins_Arr = get_bins(sideband_files)
    #return
    # Create dataframe
    cols = ["method","bintype","min","max","region","par_idx","par_name","value","error"]
    df = pd.DataFrame(columns=cols)
    
    # Loop over fitpars
    for i,bt in zip(range(len(bintype_Arr)),bintype_Arr):
        # Loop over methods
        for j,method in zip(range(len(methods)),methods):
            # Loop over regions
            for k,region in zip(range(len(regions)),regions):
                    # Loop over bins
                    for l,fitpar in zip(range(len(fitpars)),fitpars):
                        # Set TGraphErrors name
                        tgeName = method+"_"+bt+"_"+region+"_"+str(l)
                        tge=get_tge_from_asym(asym_file,m=method,bt=bt,r=region,idx=str(l))
                        # Loop over bintypes
                        for m,bl,br in zip(range(len(bins_Arr[i])-1),bins_Arr[i][:-1],bins_Arr[i][1:]):
                            value,error = get_v_e(tge,bl,br)
                            row = [method,bt,bl,br,region,l,fitpar,value,error]
                            df.loc[len(df.index)] = row
                            
    # Return dataframe
    return df
    
    #bintype = bintype_Arr[1]
    #f=get_rootfile(files = sideband_files,
    #               bintype = bintype,
    #               BIN=bins_Arr[1][1:3])
    #print(f)

# Plotting Code
---

In [7]:
def get_fmt(region):
    if(region=="sig"):
        return sig_fmt
    elif(region=="sigbg"):
        return sigbg_fmt
    elif(region=="bg"):
        return bg_fmt
    else:
        print("ERROR: get_fmt() unknown region",region,"...Aborting...")
    return

def get_label(region):
    if(region=="sig"):
        return r"$\textrm{Signal}$"
    elif(region=="sigbg"):
        return r"$\textrm{Signal+Bkg}$"
    elif(region=="bg"):
        return r"$\textrm{Bkg}$"
    else:
        print("ERROR: get_label() unknown region",region,"...Aborting...")
    return

def get_xtitle(bt):
    if(bt=="Mdihadron"):
        return "M_{h}[GeV]"
    elif(bt=="x"):
        return "x_{B}"
    elif(bt=="z"):
        return "z_{h}"
    else:
        return bt

In [ ]:
def set_xticks(axs,bintype):
    for d in dicts:
        if(d["name"]==bintype):
            axs.set_xticks(d["xticks"])

In [1]:
def plot_asym(df,dirname):
    # Grab unique values from dataframe
    bts = df["bintype"].unique()
    rs = df["region"].unique()
    
    plot_type_a(df,["sideband","splot"],["sig","sigbg","bg"],[0,1,2,3,4,5,6],dirname,"all")
    print("------------------------------------------")
    print("------------------------------------------")
    print("------------------------------------------")
    plot_type_a(df,["sideband","splot"],["sig"],[0,1,2,3,4,5,6],dirname,"sig_only")
    print("------------------------------------------")
    print("------------------------------------------")
    print("------------------------------------------")
    plot_type_a(df,["sideband","splot"],["sig"],[1,0,3],dirname,"sig_only_main")

In [13]:
def plot_type_a(df,methods,regions,par_idxs,dirname,plot_prefix):
    
    # Number of subplots for each plot
    Nsubplots = len(par_idxs)
    
    
    # Create directory 
    if(not os.path.exists("plots/asym/{}".format(dirname))):
        os.mkdir("plots/asym/{}".format(dirname))
        print("Creating directory 'plots/asym/{}'".format(dirname))
    else:
        print("Directory 'plots/asym/{}'".format(dirname),"already exists...Continuing...")
            
            
    # Set Nrows and Ncols based on number of modulations
    Nrows=-1
    Ncols=-1
    if(Nsubplots<=4):
        Nrows=1
        Ncols=Nsubplots
    else:
        Nrows=2
        Ncols=4
    
    
    # Get bin types (x, z, Mh, etc.)
    bts = df["bintype"].unique()
    
    
    # Loop over methods
    for m in methods:
        
        
        # Extract specific method from data frame
        df_0 = df.loc[(df["method"]==m)]
        
        
        # Loop over bin types
        for bt in bts:
            
            
            # Extract specific bin type from data frame
            df_1 = df_0[df_0["bintype"]==bt]
            
            
            # Create plot which will contain subplots
            fig1, axs1 = plt.subplots(Nrows,Ncols,figsize=(5*Ncols, 5*Nrows))
            
            
            # Loop over regions (sig, sigbg, bg)
            for r in regions:
                
                
                # Get formatting for the plots
                FMT = ''
                if(len(regions)==1):
                    FMT='ko'
                else:
                    FMT=get_fmt(r)
                
                # Extract specific region from data frame
                df_2 = df_1[df_1["region"]==r]

                
                # Loop over plot indexes (list of modulations)
                for par_idx,k in zip(par_idxs,range(len(par_idxs))):
                    
                    
                    # Extract specific modulation from data frame
                    df_3 = df_2[df_2["par_idx"]==par_idx]
                    
                    
                    # Set which row/column is being plotted into
                    R = int(np.floor(k/Ncols))
                    C = k % Ncols
                    
                    xArr = (df_3["min"].to_numpy() + df_3["max"].to_numpy())/2
                    yArr = df_3["value"].to_numpy()
                    yerrArr = df_3["error"].to_numpy()

                    xtitle = bt
                    ytitle = df_3["par_name"].to_list()[0]
                    
                    if(Nrows==1):
                        axs1[C].set_xlabel(r"${}$".format(get_xtitle(bt)),fontsize=18)
                        axs1[C].set_ylabel(r"$A_{LU}$",fontsize=18)
                        axs1[C].set_title(r"${}$".format(ytitle),fontsize=18)

                        axs1[C].errorbar(xArr,yArr,yerr=yerrArr,
                                           label=get_label(r),fmt=FMT, capsize=2)
                    else:
                        axs1[R,C].set_xlabel(r"${}$".format(get_xtitle(bt)),fontsize=18)
                        axs1[R,C].set_ylabel(r"$A_{LU}$",fontsize=18)
                        axs1[R,C].set_title(r"${}$".format(ytitle),fontsize=18)

                        axs1[R,C].errorbar(xArr,yArr,yerr=yerrArr,
                                           label=get_label(r),fmt=FMT, capsize=2)
            
            # --------------------------------------------------------
            # AXIS STYLING (each bullet is a special formatting)
            # --------------------------------------------------------
            # Adjust y-axes for each plot so y=0 is in the middle
            # Add a horizontal y=0 line
            # Also, remove axes for unused plots
            # Add a legend to the first plot
            # Specify method in first plot
            # Adjust location of the xtitle
            # Add minor ticks to the x and y axes
            
            unused_plot = 0
            for i,axs in zip(range(Nrows*Ncols),axs1.flatten()):
                # Delete unused subplots
                if(i>Nsubplots-1):
                    axs.set_axis_off()
                    unused_plot=i
                    continue
                if(i==0):
                    if(len(regions)>1):
                        axs.legend(loc='best')
                if(i==Ncols or (i==0 and Nrows==1)):
                    axs.annotate(m, xy=(0.5, 0.07), xycoords='axes fraction',fontsize=28,color='tan')
                ylims = axs.get_ylim()
                ymag = np.amax(np.abs(ylims))*1.1
                axs.set_ylim(-ymag,ymag)
                axs.axhline(y=0, color='0.7', linestyle='-',zorder=-1)
                axs.xaxis.set_label_coords(0.95, -0.075)
                set_xticks(axs,bt)
                axs.xaxis.set_minor_locator(AutoMinorLocator(5))
                axs.yaxis.set_minor_locator(AutoMinorLocator(5))
                axs.grid()
            
            plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.4, 
                    hspace=0.4)
            
            plt.show()
            fig1.savefig("plots/asym/{}/{}_{}_{}.pdf".format(dirname,plot_prefix,m,bt))
            fig1.savefig("plots/asym/{}/{}_{}_{}.png".format(dirname,plot_prefix,m,bt))